# ICL Implementation

## 1. Imports and Setup

In [2]:

import os
import sys
import json
import pandas as pd
import numpy as np
from pathlib import Path
from tqdm.notebook import tqdm
import time
import subprocess
from dotenv import load_dotenv
import google.generativeai as genai
from IPython.display import display, Markdown
import uuid

# Add project directory to path for imports
sys.path.append('..')

# Import our utilities
from utils.qdrant_client import get_qdrant_client
from utils.case_retrieval import retrieve_similar_cases

# Load environment variables
load_dotenv()

# Configure Google Generative AI
genai.configure(api_key=os.getenv('GOOGLE_API_KEY'))

## 2. Load Test Dataset

In [3]:
print("Loading test dataset...")
data_dir = Path("../Data/GeneratedCases")
test_file = data_dir / "test_cases.parquet"

if not test_file.exists():
    print(f"⚠️ Could not find {test_file}")
    # Find available parquet files
    parquet_files = list(data_dir.glob("*test*.parquet"))
    if parquet_files:
        print(f"Found these test parquet files: {[f.name for f in parquet_files]}")
        test_file = parquet_files[0]
        print(f"Using: {test_file}")
    else:
        # Try to use the main evaluated file and split it
        eval_file = data_dir / "evaluated_cases.parquet"
        if eval_file.exists():
            print(f"Using {eval_file} and splitting it into train/test...")
            df = pd.read_parquet(eval_file)
            from sklearn.model_selection import train_test_split
            _, test_df = train_test_split(df, test_size=0.2, random_state=42)
            print(f"Created test set with {len(test_df)} cases")
        else:
            raise FileNotFoundError(f"No test dataset files found in {data_dir}")
else:
    test_df = pd.read_parquet(test_file)
    
print(f"Loaded {len(test_df)} test cases")

# Display a sample of the test dataset
print("\nSample test case:")
sample = test_df.iloc[0]
print(f"Case ID: {sample['case_id']}")
print(f"Title: {sample['title']}")
print(f"Enhanced case (first 200 chars): {sample['enhanced_case'][:200]}...")

Loading test dataset...
Loaded 62 test cases

Sample test case:
Case ID: case-20250329-001124-k553a3
Title: Baltic Bottleneck: Navigating Congestion and Compliance in Alpha Shippings Asia-Europe Route
Enhanced case (first 200 chars): **Case Title: Baltic Bottleneck: Navigating Congestion and Compliance in Alpha Shipping's Asia-Europe Route** Alpha Shipping, a medium-sized container shipping company based in Hong Kong, specializes ...


In [4]:
test_df.head()

,case_id,title,enhanced_case,solution,file_path,enhanced_case_length,solution_length,realism_score,complexity_score,educational_value,solution_quality,overall_qualification,evaluation_summary,improvement_suggestions,case_for_embedding
183,case-20250329-001124-k553a3,Baltic Bottleneck: Navigating Congestion and C...,**Case Title: Baltic Bottleneck: Navigating Co...,## Executive Summary Alpha Shipping faces chal...,/Users/max/Documents/Code/magdeburg25/Output/G...,5821,85737,8.0,7.0,8.0,7.0,QUALIFIED,The case study presents a realistic scenario o...,[The solution could benefit from a more detail...,Baltic Bottleneck: Navigating Congestion and C...
60,case-20250329-082454-w9yo73,Gdansk Gridlock: Nordic SeaLines Struggle for ...,**Case Title: Gdansk Gridlock: Nordic SeaLines...,## Executive Summary Nordic SeaLines faces a s...,/Users/max/Documents/Code/magdeburg25/Output/G...,6872,82054,8.0,7.0,8.0,7.0,QUALIFIED,The case study presents a realistic scenario o...,[The solution could benefit from a more detail...,Gdansk Gridlock: Nordic SeaLines Struggle for ...
124,case-20250330-081631-9ahbtr,**Baltic Squeeze: Navigating Congestion and Su...,**Case Study: Baltic Squeeze: Navigating Conge...,## Executive Summary Baltic Breeze Seafoods (B...,/Users/max/Documents/Code/magdeburg25/Output/G...,7054,8970,8.0,7.0,9.0,8.0,QUALIFIED,The case study presents a realistic scenario o...,[Expand on the specific KPIs (Key Performance ...,**Baltic Squeeze: Navigating Congestion and Su...
93,case-20250329-210212-zq31iw,**OceanExs Baltic Gamble: Navigating Supply Ch...,"**Scenario:** OceanEx Logistics, a rapidly gro...",## Executive Summary OceanEx Logistics is faci...,/Users/max/Documents/Code/magdeburg25/Output/G...,7074,83931,9.0,8.0,9.0,8.0,QUALIFIED,The case study presents a highly realistic sce...,[Expand on the risk assessment for using Klaip...,**OceanExs Baltic Gamble: Navigating Supply Ch...
63,case-20250330-073349-b2bxaz,Navigating Arctic Ambitions: A Polar Silk Rout...,"**Scenario:** Klaus Weber, Head of Route Optim...",## Executive Summary Nordic Star Logistics fac...,/Users/max/Documents/Code/magdeburg25/Output/G...,8644,113848,8.0,9.0,9.0,8.0,QUALIFIED,The case study presents a realistic and comple...,[The cost-benefit analysis could be enhanced b...,Navigating Arctic Ambitions: A Polar Silk Rout...


In [5]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 62 entries, 183 to 302
Data columns (total 15 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   case_id                  62 non-null     object 
 1   title                    62 non-null     object 
 2   enhanced_case            62 non-null     object 
 3   solution                 62 non-null     object 
 4   file_path                62 non-null     object 
 5   enhanced_case_length     62 non-null     int64  
 6   solution_length          62 non-null     int64  
 7   realism_score            61 non-null     float64
 8   complexity_score         61 non-null     float64
 9   educational_value        61 non-null     float64
 10  solution_quality         61 non-null     float64
 11  overall_qualification    61 non-null     object 
 12  evaluation_summary       61 non-null     object 
 13  improvement_suggestions  61 non-null     object 
 14  case_for_embedding       62 no

## 3. Prepare Cases for Testing

In [6]:
# Prepare cases for testing - extract only what we need
test_cases = []
for _, row in test_df.iterrows():
    test_cases.append({
        "case_id": row["case_id"],
        "title": row["title"],
        "case_text": row["enhanced_case"]
    })

print(f"\nPrepared {len(test_cases)} cases for testing")


Prepared 62 cases for testing


## 4. Test Qdrant Retrieval

In [7]:
# Test the case retrieval function
print("Testing case retrieval from Qdrant...")
CASE_COLLECTION_NAME = "icl_logistics_case_examples"

# Try retrieving similar cases for a test case
test_case = test_cases[0]
print(f"Finding similar cases for: {test_case['title']}")

try:
    similar_cases = retrieve_similar_cases(
        test_case["title"] + "\n\n" + test_case["case_text"],
        num_examples=2,
        collection_name=CASE_COLLECTION_NAME
    )
    
    print(f"Found {len(similar_cases)} similar cases:")
    for i, case in enumerate(similar_cases):
        print(f"\nExample {i+1}: {case['title']} (Similarity: {case['similarity_score']:.2f})")
        print(f"Solution excerpt: {case['solution'][:150]}...")
except Exception as e:
    print(f"⚠️ Error retrieving similar cases: {e}")
    print("Check your Qdrant connection and collection name")

Testing case retrieval from Qdrant...
Finding similar cases for: Baltic Bottleneck: Navigating Congestion and Compliance in Alpha Shippings Asia-Europe Route
Found 2 similar cases:

Example 1: Jade Dragons Baltic Icebreaker: Navigating Congestion and Compliance (Similarity: 0.89)
Solution excerpt: ## Executive Summary The Jade Dragon Route is facing significant challenges due to Baltic port congestion, ice conditions, feeder service unreliabilit...

Example 2: Baltic Bottleneck: Navigating EU ETS and Customs Challenges for Golden Dragon Lines (Similarity: 0.88)
Solution excerpt: ## Executive Summary Golden Dragon Lines (GDL) faces significant challenges in its Asia-North Europe/Baltic Sea routes due to congestion, delays, esca...


In [25]:
# Print the keys in similar_cases to debug more thoroughly
print("\nDebugging similar cases structure:")
for i, case in enumerate(similar_cases):
    print(f"Case {i+1} keys: {case.keys()}")
    print(f"Sample content: {list(case.values())}")


Debugging similar cases structure:
Case 1 keys: dict_keys(['title', 'case_id', 'solution', 'similarity_score'])
Sample content: ['Jade Dragons Baltic Icebreaker: Navigating Congestion and Compliance', 'case-20250329-203120-9w5633', '## Executive Summary The Jade Dragon Route is facing significant challenges due to Baltic port congestion, ice conditions, feeder service unreliability, Suez Canal delays, increasing fuel costs, GSA performance penalties, SOLAS VI/2 compliance, customs clearance requirements, and empty container management. This solution proposes a multi-pronged approach focusing on optimizing port operations, adjusting the route, improving feeder service reliability, mitigating ice conditions, implementing fuel efficiency measures, strengthening customer relationships, and enhancing documentation control. The core strategy involves shifting the Baltic focus to Hamburg and utilizing a dedicated, reliable feeder service to serve Gdansk, Klaipeda, and other Baltic ports. Thi

## 5. Define the ICL Prompt Template

In [8]:
# This is the MedPrompt-inspired template for ICL
def create_icl_prompt(case_data, similar_cases):
    """
    Create a MedPrompt-inspired prompt for in-context learning.
    Adapts the approach from https://arxiv.org/pdf/2311.16452.pdf
    """
    # Extract the case information
    case_title = case_data["title"]
    case_text = case_data["case_text"]
    
    # Format the examples with reasoning
    formatted_examples = ""
    for i, example in enumerate(similar_cases):
        formatted_examples += f"""
## Example Case {i+1}: {example['title']}

### Problem:
{example['case_text'][:2000] if 'case_text' in example else "Case details not available"}

### Step-by-Step Solution:
{example['solution']}

"""
    
    # Create the full prompt with MedPrompt-inspired structure
    prompt = f"""# Maritime Logistics Specialist Assignment

You are an expert maritime logistics consultant specializing in international container shipping, port operations, and supply chain management. Your task is to solve a complex maritime logistics case by providing:
1. A clear understanding of the situation
2. A comprehensive analysis of the challenges
3. Strategic solutions and recommendations
4. Implementation steps and contingency plans

## Guidelines:
- Analyze every relevant aspect of the maritime logistics case
- Provide practical, actionable solutions grounded in industry best practices
- Consider regulatory compliance, cost implications, and operational feasibility
- Structure your response with clear sections and bullet points where appropriate

## Previous Similar Cases:
{formatted_examples}

## New Case to Solve:

### Case Title: {case_title}

### Problem Description:
{case_text}

## Your Solution:

Start by analyzing the situation and identifying key challenges. Then develop a comprehensive solution with specific recommendations. Use the following structure:

1. Executive Summary
2. Situation Analysis
3. Key Challenges
4. Strategic Recommendations
5. Implementation Plan
6. Contingency Options

Please provide your detailed solution now:
"""
    
    return prompt

## 6. Set Up Model


In [ ]:
# Set up the model
print("\nSetting up Gemma 3 model for ICL...")

# Directly specify the Gemma 3 model for consistency with baseline
try:
    # Use the same model as in baseline tests
    model_name = "gemma-3-27b-it"
    
    # Test if we can access the model
    model = genai.GenerativeModel(model_name=model_name)
    
    # Try a simple generation to verify access
    test_response = model.generate_content("Test")
    print(f"✅ Model {model_name} is available and working")

except Exception as e:
    print(f"⚠️ Could not access Gemma 3 model. Error: {str(e)}")
    print("Falling back to Gemini 2.0 Flash Light...")
    model_name = "gemini-2.0-flash-lite"  # Fallback option
    
    try:
        # Verify the fallback model works
        model = genai.GenerativeModel(model_name=model_name)
        test_response = model.generate_content("Test")
        print(f"✅ Fallback model {model_name} is available and working")
    except Exception as e:
        print(f"⚠️ Error with fallback model: {str(e)}")
        print("Please check your API key and access privileges")
    
print(f"Final model selection: {model_name}")

# Note for future evaluation
print("\nNote: We'll use gemini-2.5-pro-exp-03-25 as judge for evaluation later")


Setting up Gemma 3 model for ICL...
✅ Model gemma-3-27b-it is available and working
Final model selection: gemma-3-27b-it

Note: We'll use gemini-2.5-pro-exp-03-25 as judge for evaluation later


# 7. Create Results Directory and Solution Generator Function

In [28]:
# Create results directory
results_dir = Path("../Results/ICL_Gemma3")
results_dir.mkdir(parents=True, exist_ok=True)

In [29]:
# Print the keys in similar_cases to debug more thoroughly
print("\nDebugging similar cases structure:")
for i, case in enumerate(similar_cases):
    print(f"Case {i+1} keys: {case.keys()}")
    print(f"Sample content - Title: {case['title']}")
    print(f"Similarity score: {case['similarity_score']}")

# Update our ICL solution generator to match the actual data structure
def generate_icl_solution_v2(case_data, similar_cases, model_name=model_name):
    """Generate a solution using in-context learning with similar cases - adapted for our data structure"""
    
    # Create prompt with examples
    prompt = f"""Please analyze and solve the following maritime logistics case.
I'll provide you with two similar cases and their solutions for reference.

### Example Case 1: {similar_cases[0]['title']}
Solution:
{similar_cases[0]['solution'][:2000]}...

### Example Case 2: {similar_cases[1]['title']}
Solution:
{similar_cases[1]['solution'][:2000]}...

### Current Case to Solve: {case_data['title']}
{case_data['case_text']}

Please provide a comprehensive solution following a similar structure to the example solutions."""

    try:
        # Generate response using the model we verified earlier
        model = genai.GenerativeModel(model_name=model_name)
        response = model.generate_content(
            prompt,
            generation_config=genai.types.GenerationConfig(
                temperature=0.7,
                max_output_tokens=8192,
            )
        )
        
        # Return results with metadata for analysis
        return {
            "case_id": case_data["case_id"],
            "title": case_data["title"],
            "solution": response.text,
            "examples_used": [
                {"case_id": case["case_id"], "title": case["title"], "similarity_score": case["similarity_score"]} 
                for case in similar_cases
            ],
            "model": model_name,
            "prompt_length": len(prompt)
        }
    
    except Exception as e:
        print(f"Error generating solution for case {case_data['case_id']}: {e}")
        return None

# Use the updated function directly with similar_cases
test_result = generate_icl_solution_v2(test_case, similar_cases)

print(f"\nTest result for case: {test_result['title']}")
print(f"Examples used:")
for ex in test_result["examples_used"]:
    print(f" - {ex['title']} (Similarity: {ex['similarity_score']:.2f})")
print(f"Solution (first 200 chars): {test_result['solution'][:200]}...")


Debugging similar cases structure:
Case 1 keys: dict_keys(['title', 'case_id', 'solution', 'similarity_score'])
Sample content - Title: Jade Dragons Baltic Icebreaker: Navigating Congestion and Compliance
Similarity score: 0.88570917
Case 2 keys: dict_keys(['title', 'case_id', 'solution', 'similarity_score'])
Sample content - Title: Baltic Bottleneck: Navigating EU ETS and Customs Challenges for Golden Dragon Lines
Similarity score: 0.88523304

Test result for case: Baltic Bottleneck: Navigating Congestion and Compliance in Alpha Shippings Asia-Europe Route
Examples used:
 - Jade Dragons Baltic Icebreaker: Navigating Congestion and Compliance (Similarity: 0.89)
 - Baltic Bottleneck: Navigating EU ETS and Customs Challenges for Golden Dragon Lines (Similarity: 0.89)
Solution (first 200 chars): ## Executive Summary

Alpha Shipping is facing a critical challenge balancing cost, speed, and reliability in its Asia-Europe/Baltic route. The core issues are congestion in North Sea ports (Rott

# 8. Test with a Single Case


In [30]:
# Test with a single case first
print("\nTesting ICL with a single case...")
test_case = test_cases[4]

# First retrieve similar cases (this was missing)
print("Retrieving similar cases for in-context learning...")
similar_cases = retrieve_similar_cases(
    test_case["case_text"],
    num_examples=2,
    collection_name="icl_logistics_case_examples"  # Make sure this matches your collection name
)


Testing ICL with a single case...
Retrieving similar cases for in-context learning...


In [31]:
len(similar_cases)

2

In [32]:
# Check the number of retrieved similar cases
num_cases = len(similar_cases)

if num_cases < 2:
    print(f"⚠️ Only found {num_cases} similar cases, needed at least 2")
    if num_cases == 0:
        print("Cannot proceed with ICL - no examples found")
    else:
        print("Will proceed with limited examples")
elif num_cases == 2:
    print("✅ Retrieved exactly 2 similar cases - optimal for ICL")
else:
    print(f"⚠️ Retrieved {num_cases} cases - will use top 2 to maintain optimal context window size")
    similar_cases = similar_cases[:2]  # Keep only top 2 most similar cases

✅ Retrieved exactly 2 similar cases - optimal for ICL


In [33]:
# Check if we have similar cases
if not similar_cases:
    print("No similar cases retrieved!")
else:
    # Print the keys in first similar case for debugging
    print(f"Keys in similar cases: {similar_cases[0].keys()}")
    print(f"Number of similar cases retrieved: {len(similar_cases)}")

    # Make sure we have at least 2 cases for ICL
    if len(similar_cases) < 2:
        print("⚠️ Not enough similar cases for effective ICL. We need at least 2.")
        # If needed, pad with dummy cases or skip ICL
    else:
        # Keep only the top 2 most similar cases
        similar_cases = similar_cases[:2]
        
        # Generate the ICL solution directly (no need to add case_text)
        test_result = generate_icl_solution_v2(test_case, similar_cases)
        
        print(f"\nTest result for case: {test_result['title']}")
        print(f"Examples used:")
        for ex in test_result["examples_used"]:
            print(f" - {ex['title']} (Similarity: {ex['similarity_score']:.4f})")
        print(f"Solution (first 300 chars): {test_result['solution'][:300]}...")

Keys in similar cases: dict_keys(['title', 'case_id', 'solution', 'similarity_score'])
Number of similar cases retrieved: 2

Test result for case: Navigating Arctic Ambitions: A Polar Silk Route Dilemma
Examples used:
 - Arctic Peril: GlobalTechs Smartphone Shipment and the Northern Sea Route Gamble (Similarity: 0.8732)
 - Jade Dragons Baltic Icebreaker: Navigating Congestion and Compliance (Similarity: 0.8666)
Solution (first 300 chars): ## Navigating Arctic Ambitions: A Polar Silk Route Dilemma - Solution

## Executive Summary

Nordic Star Logistics faces a complex routing decision for its high-value electronics shipment from Shanghai to Rotterdam. The Red Sea instability makes the Suez Canal route increasingly risky and costly. Wh...


## 9. Save Test Result

In [34]:

# Save this test result
test_file = results_dir / f"{test_case['case_id']}_icl_test.json"
with open(test_file, 'w') as f:
    json.dump(test_result, f, indent=2)

print(f"Saved test result to {test_file}")

Saved test result to ../Results/ICL_Gemma3/case-20250330-073349-b2bxaz_icl_test.json


## 10. Prompt to Process All Cases


In [36]:
def generate_solution_with_icl(case):
    """Generate a solution for a case using in-context learning with similar cases"""
    
    # Retrieve similar cases for the current case
    similar_cases = retrieve_similar_cases(
        case["case_text"],
        num_examples=2,
        collection_name="icl_logistics_case_examples"
    )
    
    # Check number of cases retrieved
    num_cases = len(similar_cases)
    
    if num_cases < 2:
        print(f"⚠️ Case {case['case_id']}: Only found {num_cases} similar cases, needed at least 2")
        if num_cases == 0:
            print(f"⚠️ Cannot proceed with ICL for case {case['case_id']} - no examples found")
            # Return error result
            return {
                "case_id": case["case_id"],
                "title": case["title"],
                "solution": "ERROR: Could not retrieve any similar cases for ICL.",
                "examples_used": [],
                "model": model_name,
                "prompt_length": 0,
                "error": "No similar cases found"
            }
    elif num_cases > 2:
        # Keep only top 2 most similar cases
        similar_cases = similar_cases[:2]
    
    # Generate the solution using our tested function
    result = generate_icl_solution_v2(case, similar_cases)
    return result

In [37]:
# Start Caffeinate to Prevent Mac from Sleeping
print("Starting caffeinate to prevent Mac from sleeping...")
caffeinate_process = subprocess.Popen(["caffeinate", "-i", "-d", "-m"], 
                                     stdout=subprocess.DEVNULL, 
                                     stderr=subprocess.DEVNULL)

Starting caffeinate to prevent Mac from sleeping...


In [38]:
# Cell 12: Process All Cases - No Confirmation Required

# Check if caffeinate is already running, if not start it
if 'caffeinate_process' not in locals() or caffeinate_process.poll() is not None:
    print("Starting caffeinate to prevent Mac from sleeping...")
    caffeinate_process = subprocess.Popen(["caffeinate", "-i", "-d", "-m"], 
                                         stdout=subprocess.DEVNULL, 
                                         stderr=subprocess.DEVNULL)
else:
    print("Caffeinate already running...")

print(f"\nRunning ICL test on all {len(test_cases)} cases...")
results = []

try:
    for i, case in enumerate(tqdm(test_cases)):
        # Generate solution using the newly defined function
        result = generate_solution_with_icl(case)
        results.append(result)
        
        # Save the individual result
        case_file = results_dir / f"{case['case_id']}_icl.json"
        with open(case_file, 'w') as f:
            json.dump(result, f, indent=2)
        
        # Pause between requests to avoid rate limits
        if i < len(test_cases) - 1:  # Don't sleep after last case
            time.sleep(5)
    
    # Save all results to a single file
    all_results_file = results_dir / "all_icl_results.json"
    with open(all_results_file, 'w') as f:
        json.dump(results, f, indent=2)
    
    # Also save as a parquet file for easier analysis
    results_df = pd.DataFrame(results)
    results_df.to_parquet(results_dir / "icl_results.parquet")
    
    print(f"\n✅ Completed ICL testing for {len(results)} cases")
    print(f"Results saved to {results_dir}")

finally:
    # Make sure to terminate caffeinate
    if 'caffeinate_process' in locals():
        print("Terminating caffeinate - your Mac can sleep again...")
        caffeinate_process.terminate()

Caffeinate already running...

Running ICL test on all 62 cases...


  0%|          | 0/62 [00:00<?, ?it/s]

Error generating embedding: Invalid input: 'content' argument must not be empty. Please provide a non-empty value.
⚠️ Failed to generate embedding for the case
⚠️ Case 7242ee78-fcb8-4884-a7b4-b9552a9e7e3e: Only found 0 similar cases, needed at least 2
⚠️ Cannot proceed with ICL for case 7242ee78-fcb8-4884-a7b4-b9552a9e7e3e - no examples found
Error generating solution for case case-20250330-073112-b45c1i: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. [violations {
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 19
}
]
Error generating solution for case case-20250330-083816-msklnc: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. 

KeyboardInterrupt: 

In [39]:
# Check for generated ICL results
print("Checking for ICL results...")

# Define results directory path
results_dir = Path("../Results/ICL_Gemma3")

# Count and list available individual result files
icl_result_files = list(results_dir.glob("*_icl.json"))
print(f"Found {len(icl_result_files)} individual ICL result files")

if icl_result_files:
    # Show first few files
    print("Sample files:")
    for file in icl_result_files[:5]:
        print(f" - {file.name}")
    
    # Load first result file to examine structure
    with open(icl_result_files[0], 'r') as f:
        sample_result = json.load(f)
    
    print("\nSample result structure:")
    print(f"Case ID: {sample_result['case_id']}")
    print(f"Title: {sample_result['title']}")
    print(f"Examples used: {len(sample_result['examples_used'])} cases")
    for ex in sample_result['examples_used']:
        print(f" - {ex['title']} (Similarity: {ex['similarity_score']:.4f})")
    print(f"Solution length: {len(sample_result['solution'])} characters")
    print(f"First 200 chars: {sample_result['solution'][:200]}...")

Checking for ICL results...
Found 48 individual ICL result files
Sample files:
 - case-20250330-073647-bwtgdi_icl.json
 - case-20250329-194827-ftzzah_icl.json
 - case-20250329-090718-vs7b8h_icl.json
 - case-20250329-082454-w9yo73_icl.json
 - case-20250329-210312-goqhgy_icl.json

Sample result structure:
Case ID: case-20250330-073647-bwtgdi
Title: Perishable Cargo Peril: Nordic Seafoods Asian Export Challenge
Examples used: 2 cases
 - Frozen Seas, Shifting Sands: Optimizing Nordic Seafoods Asian Supply Chain (Similarity: 0.9281)
 - **Nordic Seafoods: Navigating Baltic Bottlenecks and Chinese Customs in a Perishable Supply Chain** (Similarity: 0.9216)
Solution length: 8677 characters
First 200 chars: ## Perishable Cargo Peril: Nordic Seafoods Asian Export Challenge - Solution

**Executive Summary:** Nordic Seafood Exports (NSE) faces critical challenges in its Asian seafood supply chain, resulting...


# 12. Summary and Documentation

In [40]:
# Display summary information
print("\n## ICL Implementation Summary")
print("\nThis notebook implemented a MedPrompt-inspired ICL approach for maritime logistics case solving:")
print(" - Retrieved 2 similar cases for each test case")
print(" - Used structured prompting with examples and reasoning steps")
print(" - Generated solutions with the Gemma 3 model")
print(" - Tracked which examples were retrieved for analysis")
print("\nNext steps:")
print(" - Run evaluation comparing baseline vs ICL approaches")
print(" - Analyze which types of cases benefit most from ICL")
print(" - Fine-tune the prompt template based on results")


## ICL Implementation Summary

This notebook implemented a MedPrompt-inspired ICL approach for maritime logistics case solving:
 - Retrieved 2 similar cases for each test case
 - Used structured prompting with examples and reasoning steps
 - Generated solutions with the Gemma 3 model
 - Tracked which examples were retrieved for analysis

Next steps:
 - Run evaluation comparing baseline vs ICL approaches
 - Analyze which types of cases benefit most from ICL
 - Fine-tune the prompt template based on results
